In [1]:
from diffusers import StableDiffusionPipeline
import torch

from transformers import CLIPProcessor, CLIPModel
from PIL import Image

import os

WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.2.2+cu121 with CUDA 1201 (you have 2.2.2+cu118)
    Python  3.10.11 (you have 3.10.9)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
A matching Triton is not available, some optimizations will not be enabled
Traceback (most recent call last):
  File "c:\Users\adity\AppData\Local\Programs\Python\Python310\lib\site-packages\xformers\__init__.py", line 55, in _is_triton_available
    from xformers.triton.softmax import softmax as triton_softmax  # noqa
  File "c:\Users\adity\AppData\Local\Programs\Python\Python310\lib\site-packages\xformers\triton\softmax.py", line 11, in <module>
    import triton
ModuleNotFoundError: No module named 'triton'
c:\Users\adity\AppData\Local\Programs\Python\Python310\lib\site-packages\diffuse

In [2]:
model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")
pipe.safety_checker = None

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

c:\Users\adity\AppData\Local\Programs\Python\Python310\lib\site-packages\diffusers\utils\outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [3]:
# Use this function to generate images and save them to a folder

def gen_images(prompt, folder_name):
    for i in range(100):
        image = pipe(prompt).images[0]
        image.save(f"{folder_name}/image_{i+1}.png")
    print("Done generating images")
    return True

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip_model = clip_model.to(device)

In [5]:
def get_clip_embeddings(images):
  """
    This function takes a list of images as input,
    returning a list of clip embeddings of the images.

    :param images: The input image as a string.
    :return: A list of clip embeddings.
    """
  inputs = clip_processor(images=images, return_tensors="pt", padding=True).to(device)

  with torch.no_grad():
      image_features = clip_model.get_image_features(**inputs)

  return image_features

In [11]:
def get_embedding_from_folder(folder_name):
    embeddings = []
    for img in os.listdir(folder_name):
        image = Image.open(f"{folder_name}/{img}")
        image = image.convert("RGB")
        embeddings.append(get_clip_embeddings([image]))
    return embeddings

In [12]:
# Get embedding of dog images folder
dog_folder = "dog images"
dog_embeddings = get_embedding_from_folder(dog_folder)

In [17]:
len(dog_embeddings)

100

## K-Means Clustering

In [18]:
from sklearn.cluster import KMeans
import numpy as np

# Convert the embeddings to numpy array
dog_embeddings = np.array([embedding.cpu().numpy() for embedding in dog_embeddings]).squeeze()

In [20]:
kmeans_clusters = KMeans(n_clusters=10, random_state=0, n_init="auto").fit(dog_embeddings)

In [21]:
kmeans_clusters.labels_

array([6, 4, 6, 9, 1, 1, 4, 4, 1, 8, 9, 1, 8, 7, 1, 8, 1, 1, 4, 4, 7, 8,
       1, 4, 4, 1, 8, 1, 7, 1, 8, 9, 4, 1, 2, 1, 3, 0, 3, 8, 1, 4, 8, 4,
       8, 4, 1, 4, 7, 8, 3, 4, 1, 3, 0, 9, 4, 4, 9, 9, 6, 1, 4, 4, 2, 4,
       8, 4, 4, 1, 2, 4, 1, 4, 1, 3, 4, 4, 6, 1, 8, 4, 9, 4, 6, 1, 9, 6,
       4, 1, 5, 4, 4, 0, 1, 4, 1, 9, 5, 3])

In [22]:
kmeans_clusters.cluster_centers_

array([[ 0.0940939 ,  0.14876989, -0.11178604, ...,  0.6110529 ,
        -0.13133484,  0.2154733 ],
       [ 0.09988901,  0.20128758, -0.12159383, ...,  0.6297776 ,
         0.0104358 , -0.1066051 ],
       [-0.16243565,  0.15555695, -0.07923102, ...,  0.46179855,
         0.13123655, -0.33283296],
       ...,
       [ 0.02061935, -0.03642638, -0.21178746, ...,  0.61320156,
         0.01741252,  0.07048526],
       [ 0.01687707,  0.15982442, -0.10077201, ...,  0.4878475 ,
         0.09455046, -0.05054102],
       [-0.01419756,  0.20010601, -0.00342564, ...,  0.66272056,
        -0.09371414, -0.02582545]], dtype=float32)

In [23]:
# Get different images from each cluster
def get_images_from_cluster(cluster_number, folder_name):
    cluster_images = []
    for i, label in enumerate(kmeans_clusters.labels_):
        if label == cluster_number:
            cluster_images.append(f"{folder_name}/image_{i+1}.png")
    return cluster_images

In [24]:
# Get images for all the clusters
cluster_images = []
for i in range(10):
    cluster_images.append(get_images_from_cluster(i, dog_folder))

In [25]:
cluster_images

[['dog images/image_38.png',
  'dog images/image_55.png',
  'dog images/image_94.png'],
 ['dog images/image_5.png',
  'dog images/image_6.png',
  'dog images/image_9.png',
  'dog images/image_12.png',
  'dog images/image_15.png',
  'dog images/image_17.png',
  'dog images/image_18.png',
  'dog images/image_23.png',
  'dog images/image_26.png',
  'dog images/image_28.png',
  'dog images/image_30.png',
  'dog images/image_34.png',
  'dog images/image_36.png',
  'dog images/image_41.png',
  'dog images/image_47.png',
  'dog images/image_53.png',
  'dog images/image_62.png',
  'dog images/image_70.png',
  'dog images/image_73.png',
  'dog images/image_75.png',
  'dog images/image_80.png',
  'dog images/image_86.png',
  'dog images/image_90.png',
  'dog images/image_95.png',
  'dog images/image_97.png'],
 ['dog images/image_35.png',
  'dog images/image_65.png',
  'dog images/image_71.png'],
 ['dog images/image_37.png',
  'dog images/image_39.png',
  'dog images/image_51.png',
  'dog images/